In [ ]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# configs
warnings.filterwarnings('ignore')

In [ ]:
data= pd.read_csv('global-data-on-sustainable-energy.csv')

#### Data Quality Report

##### Continuous features report 

In [ ]:
def build_continuous_features_report(data_df):

    """Build tabular report for continuous features"""

    stats = {
        "Count": len,
        "Miss %": lambda df: df.isna().sum() / len(df) * 100,
        "Card.": lambda df: df.nunique(),
        "Min": lambda df: df.min(),
        "1st Qrt.": lambda df: df.quantile(0.25),
        "Mean": lambda df: df.mean(),
        "Median": lambda df: df.median(),
        "3rd Qrt": lambda df: df.quantile(0.75),
        "Max": lambda df: df.max(),
        "Std. Dev.": lambda df: df.std(),
    }

    contin_feat_names = data_df.select_dtypes("number").columns
    continuous_data_df = data_df[contin_feat_names]

    report_df = pd.DataFrame(index=contin_feat_names, columns=stats.keys())

    for stat_name, fn in stats.items():
        # NOTE: ignore warnings for empty features
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            report_df[stat_name] = fn(continuous_data_df)

    return report_df


In [ ]:
# build continuous features report
con_report_df = build_continuous_features_report(data)
con_report_df

In [ ]:
print("categorical features are :")
data.select_dtypes(exclude="number").columns

##### Categorical features Report

In [ ]:
def build_categorical_features_report(data_df):

    """Build tabular report for categorical features"""

    def _mode(df):
        return df.apply(lambda ft: ",".join(ft.mode().to_list()))

    def _mode_freq(df):
        return df.apply(lambda ft: ft.value_counts()[ft.mode()].sum())

    def _second_mode(df):
        return df.apply(lambda ft: ",".join(ft[~ft.isin(ft.mode())].mode().to_list()))

    def _second_mode_freq(df):
        return df.apply(
            lambda ft: ft[~ft.isin(ft.mode())]
            .value_counts()[ft[~ft.isin(ft.mode())].mode()]
            .sum()
        )

    stats = {
        "Count": len,
        "Miss %": lambda df: df.isna().sum() / len(df) * 100,
        "Card.": lambda df: df.nunique(),
        "Mode": _mode,
        "Mode Freq": _mode_freq,
        "Mode %": lambda df: _mode_freq(df) / len(df) * 100,
        "2nd Mode": _second_mode,
        "2nd Mode Freq": _second_mode_freq,
        "2nd Mode %": lambda df: _second_mode_freq(df) / len(df) * 100,
    }

    cat_feat_names = data_df.select_dtypes(exclude="number").columns
    cat_data_df = data_df[cat_feat_names]

    report_df = pd.DataFrame(index=cat_feat_names, columns=stats.keys())

    for stat_name, fn in stats.items():
        # NOTE: ignore warnings for empty features
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            report_df[stat_name] = fn(cat_data_df)

    return report_df


In [ ]:
# build categorical feature report
cat_report_df = build_categorical_features_report(data)
cat_report_df

##### Checking duplicates

In [ ]:
# Check for duplicate rows
num_duplicates = data.duplicated().sum()
print("Number of Duplicate Rows:", num_duplicates)

##### Checking outliers

In [ ]:
def detect_outliers(col):
    """Detect outliers in a dataframe column

    Args:
        col (pd.Series): A dataframe column

    Returns:
        pd.Series: Outliers
    """
    Q1 = np.percentile(col, 25)
    Q3 = np.percentile(col, 75)
    IQR = Q3 - Q1
    outlier_step = IQR * 1.5

    return col[(col < Q1 - outlier_step) | (col > Q3 + outlier_step)]


In [ ]:
outliers_df = (
    data.select_dtypes("number")
    .apply(lambda col: detect_outliers(col).size)
    .rename("Num outliers")
    .to_frame()
).query("`Num outliers` > 0")

outliers_df["Percent outliers"] = outliers_df["Num outliers"] / len(data) * 100
outliers_df.sort_values(by="Percent outliers", ascending=False)


#### Data Quality Plan

#### Data Preparation

Convert the types to the desired ones

In [ ]:
def remove_functuation(density):
    return density.replace(',','')
data['Entity']=data['Entity'].astype(str)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].astype(str)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].apply(remove_functuation)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].astype(float)

##### Handling missing values

In [ ]:
#Handling missing values in continuous features
missing_con_cols = con_report_df.query("`Miss %` > 0.0").index
#print(missing_con_cols)
feature_with_null=[column for column in missing_con_cols if column not in ['Year','Primary energy consumption per capita (kWh/person)']]
#print(feature_with_null)
# Replacing values with mean
data.fillna(data[feature_with_null].mean(),inplace=True)

In [ ]:
# Handling missing values in categorical features
cat_missing = cat_report_df.query("`Miss %` > 0").index


cat_fillna_vals = data[cat_missing].mode().squeeze()
data = data.fillna(cat_fillna_vals)

In [ ]:
data.isna().sum()

In [ ]:
data.isnull().sum()

#### Handling Outliers

##### Plot heatmap